<a href="https://colab.research.google.com/github/Tiru-Kaggundi/Adv_ML_project/blob/main/Tiru_PoS_Tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install indic-nlp-library
!pip install torch
!pip install torchtext
!pip install spacy
!pip install torchdata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchtext
from torchtext import data
from torchtext import datasets

from collections import Counter
from torchtext.vocab import vocab

import numpy as np
import random

In [4]:
import sys
import os
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)
%cd /content/gdrive/MyDrive/AML_Project/
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/AML_Project"

Mounted at /content/gdrive
/content/gdrive/MyDrive/AML_Project


In [28]:
import pandas as pd
import re

df = pd.DataFrame(columns=['sentence'])
with open('/content/gdrive/MyDrive/AML_Project/tagger_data/newTrainH1.txt',encoding="utf8") as fp:
   line = fp.readline()
   cnt = 0
   while line:
       #print("Line {}: {}".format(cnt, line.strip()))
       line = fp.readline()
       line = line.replace('_', '')
       df.loc[cnt, ['sentence']] = line
       cnt += 1 
       

def split_for_tagger(sentence):
    res = re.findall(r'\[.*?\]', sentence)
    RES= []
    for elem in res:
        #print(elem)
        a = elem.replace('[','').replace(']','')
        b = a.replace(" ","")
        RES.append(b)
    return RES

def split_for_word(sentence):
    string = re.sub("\[.*?\]","",sentence)
    c= string.split()
    return c
    
df['pos'] = df['sentence'].apply(split_for_tagger)
df['clean_sentence'] = df['sentence'].apply(split_for_word)
    
df.drop(['sentence'], axis=1, inplace= True)


In [29]:
df

,pos,clean_sentence
0,"[JJ, NN, PSP, NN, SYM, PRP, NN, SYM, NN, PSP, ...","[पुरानी, पीढ़ी, के, भारतीय, --, हमारे, पिता-पित..."
1,"[JJ, SYM, JJ, SYM, NST, VM, PSP, JJ, NN, PSP, ...","[बदमिज़ाज, ,, चिड़चिड़े, ,, दूर-दूर, रहने, वाले, ..."
2,"[PRP, NN, PSP, INTF, JJ, NN, PRP, NN, VM, VAUX...","[अपनी, ऐंठ, में, थोड़ा-बहुत, भद्र, पीढ़ी-दर-पीढ़ी..."
3,"[PRP, NN, VM, CC, CC, PRP, SYM, NN, SYM, CC, S...","[मुझे, विश्वास, है, कि, अगर, उनकी, `, खुशमिज़ाज..."
4,"[CC, PRP, PRP, PRP, NN, PSP, NN, VM, VAUX, VAU...","[और, जब, मैं, अपने, पिता, की, बात, कर, रहा, हू..."
...,...,...
2595,"[NNP, PSP, VM, SYM, SYM, NNP, CC, NNP, JJ, JJ,...","[शिराक़, ने, कहा, ,, "", यूरोप, और, अमरीका, सच्..."
2596,"[PRP, JJ, NN, PSP, QO, NN, NN, NNP, PSP, NNP, ...","[अपने, यूरोपीय, दौरे, के, दूसरे, दिन, राष्ट्रप..."
2597,"[NNP, PSP, DEM, NNP, VM, PRP, DEM, NN, NNP, NN...","[बुधवार, को, वे, जर्मनी, जाएँगे, जहाँ, वे, चां..."
2598,"[NN, PSP, JJ, NN, NNP, PSP, NNP, NNP, NNP, VM,...","[दौरे, के, अंतिम, दिन, गुरूवार, को, जॉर्ज, बुश..."


In [45]:
df["example"] = df.apply(lambda row: [row.clean_sentence, row.pos], axis=1)

In [46]:
HINDIPOS = df[["example"]]
HINDIPOS

,example
0,"[[पुरानी, पीढ़ी, के, भारतीय, --, हमारे, पिता-पि..."
1,"[[बदमिज़ाज, ,, चिड़चिड़े, ,, दूर-दूर, रहने, वाले,..."
2,"[[अपनी, ऐंठ, में, थोड़ा-बहुत, भद्र, पीढ़ी-दर-पीढ़..."
3,"[[मुझे, विश्वास, है, कि, अगर, उनकी, `, खुशमिज़ा..."
4,"[[और, जब, मैं, अपने, पिता, की, बात, कर, रहा, ह..."
...,...
2595,"[[शिराक़, ने, कहा, ,, "", यूरोप, और, अमरीका, सच..."
2596,"[[अपने, यूरोपीय, दौरे, के, दूसरे, दिन, राष्ट्र..."
2597,"[[बुधवार, को, वे, जर्मनी, जाएँगे, जहाँ, वे, चा..."
2598,"[[दौरे, के, अंतिम, दिन, गुरूवार, को, जॉर्ज, बु..."


In [47]:
train_iter = datasets.UDPOS(split = 'train')

for i, example in enumerate(train_iter):
    print(f'The sentence for Example {i} ---')
    print(len(example[0]), len(example[1]), len(example[2]))
    if i == 1: break

The sentence for Example 0 ---
29 29 29
The sentence for Example 1 ---
18 18 18


In [48]:
from torch.utils.data import Dataset, DataLoader
# create custom dataset class
class CustomTextDataset(Dataset):
    def __init__(self, df):
        self.examples = df.iloc[:,0]
 
    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        if idx >= self.__len__():
          raise StopIteration
        example = self.examples[idx]
        return example

In [49]:
W = 1
WINDOW_SIZE = (2 * W + 1)

SENT_START_WORD = '<s>'
SENT_END_WORD = '</s>'
SENT_START_TAG = '<STAG>'
SENT_END_TAG = '<ETAG>'



def add_sent_start_end(data_iter, w):
    for (words, ud_tags) in data_iter:
        new_words = [SENT_START_WORD] * w + words + [SENT_END_WORD] * w
        new_ud_tags = [SENT_START_TAG] * w + ud_tags + [SENT_END_TAG] * w
        yield(new_words, new_ud_tags)
        
def create_windows(data_iter, w):
    window_size = 2*w + 1
    for (words, ud_tags) in data_iter:
        words_zip = zip(*[words[i:] for i in range(window_size)])
        ud_zip = zip(*[ud_tags[i:] for i in range(window_size)])
        ## ADD YOUR CODE ABOVE
        for word_sseq, ud_sseq in zip(
                words_zip, ud_zip):
            yield(word_sseq, ud_sseq)
            
def preprocess_data_seq(data_iter, w):
    ## MISSING PART: ADD YOUR CODE BELOW
    data_iter_new = add_sent_start_end(data_iter, w)
    rv = create_windows(data_iter_new, w) 
    return rv

In [50]:
# It is important to recreate the training set afresh 
# each time they are used, since they are Python iterators, and
# once use, cannot be reused.  One could, however, cast them into
# a list, which would store them permanently.

train_iter_0 = CustomTextDataset(HINDIPOS)
train_iter_vocab = preprocess_data_seq(train_iter_0, 1)

counter_words = Counter()
counter_ud = Counter()
for (text, pos_ud) in train_iter_vocab:
    counter_words.update(text)
    counter_ud.update(pos_ud)
vocab_words = vocab(counter_words,  specials = ['<unk>'], special_first = True)    
vocab_words.set_default_index(0)
vocab_ud = vocab(counter_ud)
vocab_ud.set_default_index(-1)

In [51]:
len(vocab_words)

7318

In [52]:
len(vocab_ud)

27

In [54]:
vocab_ud.get_stoi()

{'<ETAG>': 8,
 '<STAG>': 0,
 'CC': 11,
 'DEM': 16,
 'INJ': 23,
 'INTF': 10,
 'JJ': 1,
 'JJC': 26,
 'NEG': 17,
 'NN': 2,
 'NNC': 21,
 'NNP': 19,
 'NNPC': 22,
 'NST': 9,
 'PRP': 5,
 'PSP': 3,
 'QC': 15,
 'QF': 12,
 'QO': 20,
 'QW': 18,
 'RB': 13,
 'RDP': 24,
 'RP': 14,
 'SYM': 4,
 'UT': 25,
 'VAUX': 7,
 'VM': 6}